In [72]:
import sys
import glob
import os
import platform
import math
from scipy.optimize import fsolve
import pandas as pd
from pandas import ExcelWriter
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import matplotlib.pyplot as plt
import numpy as np

# *MARKOWITZ EFFICIENT FRONTIER*

# Only calculate efficient frontier WITH covariance matrix

In [160]:
# ARIZONA

# datapath = os.path.join('/Users', 'jacob.chuslo', 'Desktop', 'work', 'projects', 'jp_proj')
# file = os.path.join(datapath, "2015_6_CAPM_ArizonaSRS.xlsx")
file = "2015_6_CAPM_ArizonaSRS.xlsx"
os.chdir(os.path.join('/Users', 'jacob.chuslo', 'Desktop', 'work', 'projects', 'jp_proj'))
mean = pd.read_excel(file,sheet_name='ExpectedReturn')
var = pd.read_excel(file,sheet_name='StandardDeviation')
corr = pd.read_excel(file,sheet_name='Correlation', index_col=0)

In [161]:
del corr.index.name
corr = corr.rename(columns={'1.1':'1'})

In [162]:
mean = mean[['Asset Class','Geometric Nominal Return','Period (in Years)']]
mean.columns = ['class','return','period']
mean = mean[mean.period == 30]
mean = mean.sort_values(by=['period','class'])
var.columns = ['class','sigma','period']
var = var[var.period == 30]
var = var.sort_values(by=['period','class'])

In [163]:
mean = mean[~mean['class'].isin([2,23,24,25,999]) ]
var = var[~var['class'].isin([2,23,24,25,999]) ]
var = var[['sigma']]
var = var**2
var = var.reset_index(drop=True)

In [164]:
var = var.to_numpy()
cov_pre = var * np.transpose(var)
cov_pre = np.sqrt(cov_pre)
corr = corr.to_numpy()
cov_pre = pd.DataFrame(data=cov_pre)
cov = corr * cov_pre
cov = pd.DataFrame(data=cov)

In [165]:
# cov = cov.fillna(value=0)
cov

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000110,0.003025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000075,0.002681,0.005625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000175,0.003465,0.003938,0.004900,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,0.002681,0.003375,0.003675,0.005625,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.000065,0.001430,0.005363,0.002730,0.001950,0.016900,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.000090,0.002475,0.003375,0.002835,0.002700,0.001170,0.008100,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.000075,0.002200,0.002438,0.002450,0.002438,0.001300,0.002700,0.002500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.000060,0.002310,0.005850,0.002940,0.002700,0.009360,0.002700,0.002100,0.01440,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.000075,0.002475,0.006750,0.002625,0.002812,0.011700,0.004050,0.001875,0.01440,0.022500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# By asset class/period

In [ ]:
datapath = os.path.join('/Users', 'jacob.chuslo', 'Desktop', 'work', 'projects', 'jp_proj')
file = os.path.join(datapath, "AlaskaTRS.xlsx")
os.chdir(os.path.join('/Users', 'jacob.chuslo', 'Desktop', 'work', 'projects', 'jp_proj'))
mean = pd.read_excel("AlaskaTRS.xlsx",sheet_name='ExpectedReturn')
variance = pd.read_excel("AlaskaTRS.xlsx",sheet_name='StandardDeviation')

In [ ]:
mean = mean[['Asset Class','Arithmetic Nominal Return','Period (in Years)']]
# mean = mean.drop(columns=['Geometric Real Return','Arithmetic Real Return'])
mean.columns = ['class','return','period']
mean = mean.dropna()
mean = mean.sort_values(by=['class'])
variance.columns = ['class','sigma','period']
mean = mean[mean.period == 1]
variance = variance[variance.period == 1]
variance = variance.sort_values(by=['class'])

In [ ]:
frontier = pd.merge(mean, variance, on=['class','period'])
# frontier = frontier[['class','return','sigma']]
# frontier

In [ ]:
plt.style.use('seaborn-dark')
frontier.plot.scatter(x='sigma', y='return', figsize=(10, 8), grid=True)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

# All data together

In [ ]:
datapath = os.path.join('/Users', 'jacob.chuslo', 'Desktop', 'work', 'projects', 'jp_proj')
file = os.path.join(datapath, "AlaskaTRS.xlsx")
os.chdir(os.path.join('/Users', 'jacob.chuslo', 'Desktop', 'work', 'projects', 'jp_proj'))
mean = pd.read_excel("AlaskaTRS.xlsx",sheet_name='ExpectedReturn')
variance = pd.read_excel("AlaskaTRS.xlsx",sheet_name='StandardDeviation')

In [ ]:
mean = mean.drop(columns=['Geometric Real Return','Arithmetic Real Return'])
mean = np.where(np.isnan(mean), 0, mean)
mean = pd.DataFrame(data=mean)
mean.columns = ['class','return_a','return_g','period']
# mean.head()

In [ ]:
# You can also remake this loop to go by row, and if the program finds a non-NaN value, to keep it
for i in range(0,len(mean)):
    if mean.loc[i,'return_g'] == 0:
        mean.loc[i,'return_g'] = mean.loc[i,'return_a']
    else:
        pass

In [ ]:
mean = mean.drop(columns='return_a')
mean = mean.rename(columns={'return_g':'return'})

In [ ]:
mean = mean.sort_values(by=['period','class'])
variance.columns = ['class','sigma','period']
variance = variance.sort_values(by=['period','class'])

In [ ]:
frontier = pd.merge(mean, variance, on=['class','period'])

In [ ]:
plt.style.use('seaborn')
frontier.plot.scatter(x='sigma', y='return', figsize=(10, 8), grid=True)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
# Monte Carlo simulation of portfolio compositions
rets = []
vols = []

for w in range(50000):
    weights = np.random.random(len(frontier))
    weights /= sum(weights)
    r = weights * frontier['return']
    v = weights * frontier['sigma']
    rets.append(r)
    vols.append(v)

rets = np.array(rets)
vols = np.array(vols)

In [ ]:
rets = rets.flatten()
vols = vols.flatten()
frontier2 = pd.DataFrame({'return':rets, 'sigma':vols})

In [ ]:
plt.style.use('seaborn')
frontier2.plot.scatter(x='sigma', y='return', figsize=(10, 8), grid=True)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()